In [1]:
from iroha import IrohaCrypto
from iroha.ed25519 import H
import integration_helpers
from iroha.primitive_pb2 import can_set_my_account_detail
import json
from iroha_helper import *
from new_helper import *
from super_helper import *
from ipfs_functions import *
from loguru import logger

In [2]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 2

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

#for the index system
index_path = "indexdir"
index = open_dir(index_path)


In [3]:
#Manually resets the index on execution
# logger.info(recreate_index()) 

1 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [4]:
hash = create_detail_contract()
integration_helpers.get_engine_receipts_result(hash)

2025-01-14 21:57:53.179 | INFO     | integration_helpers:tracer:34 - 	Entering "create_detail_contract"
2025-01-14 21:57:53.186 | INFO     | iroha_helper:create_detail_contract:55 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-14 21:57:53.187 | INFO     | iroha_helper:create_detail_contract:55 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 21:57:54.859 | INFO     | iroha_helper:create_detail_contract:55 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 21:57:54.890 | INFO     | iroha_helper:create_detail_contract:55 - ('COMMITTED', 5, 0)
2025-01-14 21:57:54.892 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-14 21:57:54.893 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-14 21:57:54.913 | INFO     | integration_helpers:get_engine_receipts_result:92 - 
2025-01-14 21:57:54.915 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_result"


2025-01-14 21:29:16.735 | INFO     | iroha_helper:create_detail_contract:55 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 21:29:16.764 | INFO     | iroha_helper:create_detail_contract:55 - ('COMMITTED', 5, 0)
2025-01-14 21:29:16.766 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-14 21:29:16.768 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-14 21:29:16.789 | INFO     | integration_helpers:get_engine_receipts_result:92 - 
2025-01-14 21:29:16.790 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_result"


2 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` with a entry number related to an existing object in `datasets/accounts.json` and `datasets/projects.json`

4 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [5]:
# Example execution of the previous snippet
address = integration_helpers.get_engine_receipts_address(hash)

# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
project_account = project_accounts[json_ld_index]

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

2025-01-14 21:57:54.930 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-14 21:57:54.952 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-14 21:57:54.954 | INFO     | iroha_helper:tracer:244 - 	Entering "set_account_detail"
2025-01-14 21:57:54.975 | INFO     | iroha_helper:set_account_detail:296 - None
2025-01-14 21:57:54.977 | INFO     | iroha_helper:set_account_detail:298 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 21:57:57.919 | INFO     | iroha_helper:set_account_detail:298 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 21:57:57.946 | INFO     | iroha_helper:set_account_detail:298 - ('COMMITTED', 5, 0)
2025-01-14 21:57:57.948 | INFO     | iroha_helper:tracer:246 - 	Leaving "set_account_detail"
2025-01-14 21:57:57.950 | INFO     | iroha_helper:tracer:244 - 	Entering "set_account_detail"
2025-01-14 21:57:57.965 | INFO     | iroha_helper:set_account_detail:296 - None
2025-01-14 21:57:57.9

2025-01-14 21:29:19.789 | INFO     | iroha_helper:set_account_detail:298 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 21:29:19.814 | INFO     | iroha_helper:set_account_detail:298 - ('COMMITTED', 5, 0)
2025-01-14 21:29:19.817 | INFO     | iroha_helper:tracer:246 - 	Leaving "set_account_detail"
2025-01-14 21:29:19.819 | INFO     | iroha_helper:tracer:244 - 	Entering "set_account_detail"
2025-01-14 21:29:19.835 | INFO     | iroha_helper:set_account_detail:296 - None
2025-01-14 21:29:19.838 | INFO     | iroha_helper:set_account_detail:298 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 21:29:22.838 | INFO     | iroha_helper:set_account_detail:298 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 21:29:22.865 | INFO     | iroha_helper:set_account_detail:298 - ('COMMITTED', 5, 0)
2025-01-14 21:29:22.867 | INFO     | iroha_helper:tracer:246 - 	Leaving "set_account_detail"
2025-01-14 21:29:22.871 | INFO     | iroha_helper:update_user_account_link:160 - Updated user account jolly_car

3 - Queries Iroha 1 for User account and checks its values

In [6]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail',account_id=user_account['account_id'])
# logger.info(query)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
# logger.info(response)

user_data = response.account_detail_response
user_details = user_data.detail

logger.info(f'User Account id = {user_account}, {user_details}')

2025-01-14 21:58:01.044 | INFO     | __main__:<module>:11 - User Account id = {'account_id': 'jolly_carson@test'}, { "admin@test" : { "linked_project" : "60032@test", "user_json_ld_cid" : "QmaooJHb4vCziESs7MqcC4LhnR7RfMMhG3iQvcpUyxrUHU" } }


6 - Queries the user account, locates the project id, queries the project account, gets the metadata and files from IPFS.

In [7]:
# Process the account details response
user_details_dict = json.loads(user_details)  # Convert the string to a JSON object
logger.info(user_details_dict)

# Now you can access the specific key like this
project_id = user_details_dict["admin@test"]["linked_project"]
logger.info(project_id)

2025-01-14 21:58:01.057 | INFO     | __main__:<module>:3 - {'admin@test': {'linked_project': '60032@test', 'user_json_ld_cid': 'QmaooJHb4vCziESs7MqcC4LhnR7RfMMhG3iQvcpUyxrUHU'}}
2025-01-14 21:58:01.060 | INFO     | __main__:<module>:7 - 60032@test


In order to star a new notebook from here, the only dependency is `project_id`

In [9]:
import pickle

# Save project_id to a file

with open("temp/project_id.pkl", "wb") as f:
    pickle.dump(project_id, f)
